# Исследование надёжности заёмщиков

Заказчик, в лице кредитного отдела банка, хочет выяснить: влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

План действий:
- Откроем и изучим файл с данными
- Предобработаем данные для улучшения качества проводимого анализа
- Ответим на главные вопросы исследования
- Сформулируем общий вывод

Результаты исследования помогут в построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Оглавение
1. [Открытие данных и изучение общей информации](#step1)
2. [Предобработка данных](#step2)
    * [Обработка пропусков](#fillna) 
    * [Замена типа данных](#type_replace)
    * [Поиск и обработка артефактов](#artefacts)
    * [Обработка дубликатов](#duplicates)
    * [Лемматизация](#lemm)
    * [Категоризация данных](#categorize)
3. [Ответы на вопросы](#step3)
    * [Есть ли зависимость между наличием детей и возвратом кредита в срок?](#question1)
    * [Есть ли зависимость между семейным положением и возвратом кредита в срок?](#question2)
    * [Есть ли зависимость между уровнем дохода и возвратом кредита в срок?](#question3)
    * [Как разные цели кредита влияют на его возврат в срок?](#question4)
4. [Общий вывод](#end)

<a id="step1"></a>
## Шаг 1. Открываем файл с данными и изучаем общую информацию. 


In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Вывод

Данные представляют собой таблицу. Каждая строка состоит из наблюдений за одним из клиентов банка по двеннадцати признакам. Названия столбцов и индексов оптимальные, поэтому переименовывать их не стоит. 

В двух столбцах есть отсутствующие значения.

Столбцы education, family_status, gender, income_type и purpose заполнены строковыми значениями.

Столбцы children, dob_years, education_id, family_status_id заполнены численными значениями.

Значения в столбце debt заполнены по бинарному типу, где 0 означет 'нет', а 1 - 'да'.

<a id="step2"></a>
## Шаг 2. Предобработка данных

<a id="fillna"></a>
## Обработка пропусков

Посчитаем количество пропусков в каждом столбце.

In [2]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропуски существуют только в двух столбцах, причем в обоих их одинаковое количество.
Рассмотрим взаимосвязанно ли это.

In [3]:
data[ (data['days_employed'].isna()) & (data['total_income'].notna()) ]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Пытаясь найти строки в которых есть пропущенные значения только в одном из двух столбцов получили пустую таблицу. Это свидетельствует о том, что каждая строка, в которой есть пропуск хотябы в одном столбце, значение во втором так же отсутствует. Значит предположение о том, что пропуски взаимосвязаны, подтвердилось.

Самые очевидные причины таких пропусков можно разделить на две группы:

1. Человек не работал всю свою жизнь.
     Проблемы со здоровьем можно сразу исключить, так как при инвалидности платят пособие, а это уже задокументированный ежемесячный доход.
     Остается лишь содержание за чужой счет.
     
2. Человек никогда не работал официально и соответственно не может никак доказать свой доход.

Так же можно исключить нежелание заполнять данные, так как их указание только повышает шансы одобрения кредита - то за чем и пришел клиент.



Заполним пропуски средними значениями.
Чтобы повысить точность, среднее будем находить для каждой опорной группы. Так для столбца со стажем это будет возрас клиента, а для дохода - тип занятости.
После замены проверим что пропусков не осталось.

In [4]:
# методом transform() присвоим медианные значения, вычисленные для каждой из групп, и заполним ими пропуски 
# с помощью метода fillna()
data['days_employed'] = data['days_employed'].fillna(data.groupby('dob_years')['days_employed'].transform('median'))
data['total_income'] = data['total_income'].fillna(data.groupby('income_type')['total_income'].transform('median'))
data['days_employed'].isna().sum()

0

### Вывод

Исключена возможность случайных ошибок при обработке данных. Отсутствующие данные заменены на подходящие значения.

<a id="type_replace"></a>
## Замена типа данных

В столбцах days_employed и total_income значения представлены типом float. Так как дробная часть фактически никак не влияет на анализ, как в случае с ежемесячным доходом(дробная часть - копейки), так и в стаже(дробная часть - кол-во часов, до 24-х), для наглядности и удобства приведем эти столбцы к целочисленному типу.

In [5]:
data['days_employed'] = data['days_employed'].astype('int') # Для изменения типа данных был выбран метод astype() потому что 
data['total_income'] = data['total_income'].astype('int')   # до этого все значения уже соответствовали типу float
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


В столбце days_employed большая часть данных указана со знаком минус. Невозможно точно сказать: ошибка ли это при заполнении или, возможно, отсчет от какой-то даты. Дополнительной технической документации на этот счет не предоставленно. Эта метрика не представляет ценности в нашем исследовании, поэтому оставим колонку без дальнейших изменнеий.

### Вывод

Столбцы состоящие из вещественных чисел заменены на целочисленные.

<a id="artefacts"></a>
## Поиск и обработка артефактов

Для начала приведем значения всех столбцов, в которых данные представлены в виде строк, к одному(нижнему) регистру. И убедимся, что нет одинаковых по смыслу значений, которые записаны разными способами.

In [6]:
data['education'] = data['education'].str.lower()
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [7]:
data['family_status'] = data['family_status'].str.lower()
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

In [8]:
data['income_type'] = data['income_type'].str.lower()
data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [9]:
data['purpose'] = data['purpose'].str.lower()
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

В столбце purpose действительно много схожих по смыслу строк, но ими мы займемся при лемматизации.

In [10]:
data['gender'] = data['gender'].str.lower()
data['gender'].unique()

array(['f', 'm', 'xna'], dtype=object)

В столбце gender обнаружено непонятное значение пола клиента 'xna'. Посмотрим сколько же клиентов решило указать свой пол таким образом.

In [11]:
data['gender'].value_counts()

f      14236
m       7288
xna        1
Name: gender, dtype: int64

Такой клиент всего один. Его тяжело правильно интерпретировать и заменить каким-то из двух остальных значений. Поэтому, самым логичным способом будет удаление такого наблдюдения.

In [12]:
data = data[data['gender'] != 'xna']

Первым столбцом, сотоящем из чисел, будем рассматривать столбец dob_years.
Посмотрим на его уникальные значения и отсортируем для того, чтобы бы удобнее выявить артефакты.

In [13]:
# так же осортируем для наглядности
sorted(data['dob_years'].unique()) 

[0,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75]

Сразу же первым значением выпадает ноль. Понятно, что это ошибка. Она может иметь технический характер, а может быть люди, не захотевшие указать свой возраст решили так проявить свою оригинальеность(это, конечно, маловероятно, потому что для взятия кредита необходимы паспортные данные, которые включают в себя точный возраст). 

Так как разброс возрастов очень велик (то 19 до 75 лет), заменим нули медианным значением. После этого проверим, что значений равных нулю не осталось.

In [14]:
med = data['dob_years'].median()

def fill_median(ceil):
    if ceil == 0:
        return med

# с помощью созданной функции, котороя при встрече с нулевым значение возвращает найденную медиану,
# заполним нулевые значения
data['dob_years'] = data['dob_years'].apply(fill_median)
data[data['dob_years'] == 0].count()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Теперь проверим столбец children.

In [15]:
data['children'].value_counts()

 0     14148
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Значения -1 и 20 в контексте детей довольно сомнительны. Подобных данных меньше процента, поэтому дабы они не исказили результаты анализы, мы можем отказаться от них.

In [16]:
data = data[ (data['children'] != -1) & (data['children'] != 20)]

### Вывод

Все строчные данные приведены к одному регистру. В столбцах нужных для ответов на вопросы обработаны артефакты.

<a id="duplicates"></a>
## Обработка дубликатов

Теперь узнаем присутствуют ли в данных повторения.

In [17]:
data.duplicated().sum() #метод duplicated() найдет каждую повторяющуююся строку, а sum() посчитает их количество

74

Избавимся от абсолютно идентичных строк, так как они могут сильно испортить анализ. Вероятнее всего они вызваны техническими сбоями или человеческим фактором. 

In [18]:
data = data.drop_duplicates().reset_index(drop=True) 
# метод drop_duplicates() удаляет абсолютно идентичные строки в DataFrame, а метод reset_index() восстановит
# нужный порядок индексов

### Вывод

Удалены дубликаты, которые могли неблагоприятно повлиять на результаты дальнейшего аналаиза.

<a id="lemm"></a>
## Лемматизация

Найдем основные причины обращения за кредитом.

Для начала посмотрим все уникальные значения.

In [19]:
data['purpose'].value_counts()

свадьба                                   790
на проведение свадьбы                     763
сыграть свадьбу                           759
операции с недвижимостью                  672
покупка коммерческой недвижимости         658
покупка жилья для сдачи                   649
операции с жильем                         646
операции с коммерческой недвижимостью     645
жилье                                     641
покупка жилья                             640
покупка жилья для семьи                   637
недвижимость                              631
строительство собственной недвижимости    628
операции со своей недвижимостью           623
строительство жилой недвижимости          619
строительство недвижимости                619
покупка своего жилья                      619
покупка недвижимости                      615
ремонт жилью                              604
покупка жилой недвижимости                602
на покупку своего автомобиля              504
заняться высшим образованием      

Выделим основные леммы. 

Просуммируем все строки и применим к ним методы лемматизации из библиотеки pymystem3.

In [20]:
from pymystem3 import Mystem
m = Mystem()
from collections import Counter

Counter(m.lemmatize((data['purpose'] + ', ').sum()))  # для того, чтобы при суммировании слова не склеивалисть, добавим
                                                      # к каждому значению в столбце символы ', '
                                                      # затем объеденним их всех в одну строку из которой и извлечем леммы 

Counter({'покупка': 5864,
         ' ': 33372,
         'жилье': 4436,
         ', ': 21326,
         'приобретение': 459,
         'автомобиль': 4279,
         'дополнительный': 899,
         'образование': 3988,
         'сыграть': 759,
         'свадьба': 2312,
         'операция': 2586,
         'с': 2899,
         'на': 2207,
         'проведение': 763,
         'для': 1286,
         'семья': 637,
         'недвижимость': 6312,
         'коммерческий': 1303,
         'жилой': 1221,
         'строительство': 1866,
         'собственный': 628,
         'подержать': 842,
         'свой': 2219,
         'со': 623,
         'заниматься': 904,
         'сделка': 936,
         'получение': 1309,
         'высокий': 1367,
         'подержанный': 110,
         'профильный': 432,
         'сдача': 649,
         'ремонт': 604,
         ', \n': 1})

### Вывод

Из результата видны основные цели обращения в банк: жилье/недвижимость, автомобиль, образование, свадьба и ремонт.

<a id="categorize"></a>
## Категоризация данных

 **Категоризация по целям обращения**
 
 Присвоим каждой категории из найденных целей обращения за кредитом свой номер и добавим новый столбец с ним.

In [21]:
def choose_purpose_category(s):
    dct = {'жилье': 'недвижимость', 'недвижимость': 'недвижимость', 'автомобиль': 'автомобиль',
           'образование': 'образование', 'свадьба': 'свадьба', 'ремонт': 'ремонт'}
    s = m.lemmatize(s)
    for element in s:
        if element in dct:
            return dct[element]
# в функцию передается строка(значение одной ячейки) она возвращает значение из словаря(номер категории), если
# какое-либо слово в этой строке соответствует ключу словаря

data['purpose_category'] = data['purpose'].apply(choose_purpose_category)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,-8437,NaN,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,недвижимость
1,1,-4024,NaN,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,-5623,NaN,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,недвижимость
3,3,-4124,NaN,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,NaN,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,свадьба


Посмортим какое количество клиентов соответствует созданным группам.

In [22]:
# для наглядности выведем методом sort_values значения в порядке убывания   
data['purpose_category'].value_counts().sort_values(ascending=False)  

недвижимость    10144
автомобиль       4279
образование      3988
свадьба          2312
ремонт            604
Name: purpose_category, dtype: int64

Хоть клиентов и больше 20-ти тысяч, люди обращаются за кредитом в основном только в случае больших по сумме покупок. И чем дороже цель, тем чаще они обращаются.

В таблице есть два столбца: education_id и family_status_id, которые отображают категории значений в столбцах education и family_status соответственно. Мы не можем быть уверены, что в кажом наблюдении стоят соответствующие значения. Из-за технических сбоев или человеческого фактора возможны неправильные присвоения категорий. Просматривать каждого клиента было бы нерационально, поэтому мы заново категоризируем оба столбца, создав для каждого свой словарь соответствия. 

**Категоризация по уровню образования**

Сначала посмотрим на уникальные значения столбца education.

In [23]:
data['education'].value_counts()

среднее                15074
высшее                  5225
неоконченное высшее      740
начальное                282
ученая степень             6
Name: education, dtype: int64

Всего 5 уникальных значенией. Нет повторяющихся по смыслу. Присвоим каждой категории свой номер и перезаполним значения в столбце education_id.

In [24]:
def choose_education_id(s):
    dct = {'среднее': 1, 'высшее': 2, 'неоконченное высшее': 3, 'начальное': 4, 'ученая степень': 5}
    if s in dct:
        return dct[s]
# в функцию передается значение, если оно соответствует элементу словаря, то функция
# возвращает соответствующее значение(номер категории)
    
data['education_id'] = data['education'].apply(choose_education_id)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,-8437,NaN,высшее,2,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,недвижимость
1,1,-4024,NaN,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,-5623,NaN,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,недвижимость
3,3,-4124,NaN,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,NaN,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,свадьба


**Категоризация по семейному статусу**

Посморим на уникальные значения столбца family_status.

In [25]:
data['family_status'].value_counts()

женат / замужем          12259
гражданский брак          4132
не женат / не замужем     2796
в разводе                 1189
вдовец / вдова             951
Name: family_status, dtype: int64

Всего 5 уникальных значенией. Нет повторяющихся по смыслу. Присвоим каждой категории свой номер и перезаполним значения в столбце education_id.

In [26]:
def choose_family_status_id(s):
    dct = {'женат / замужем': 1, 'гражданский брак': 2, 'не женат / не замужем': 3, 'в разводе': 4, 'вдовец / вдова': 5}
    if s in dct:
        return dct[s]
# в функцию передается значение, если оно соответствует элементу словаря, то функция
# возвращает соответствующее значение(номер категории)
    
data['family_status_id'] = data['family_status'].apply(choose_family_status_id)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,-8437,NaN,высшее,2,женат / замужем,1,f,сотрудник,0,253875,покупка жилья,недвижимость
1,1,-4024,NaN,среднее,1,женат / замужем,1,f,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,-5623,NaN,среднее,1,женат / замужем,1,m,сотрудник,0,145885,покупка жилья,недвижимость
3,3,-4124,NaN,среднее,1,женат / замужем,1,m,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,NaN,среднее,1,гражданский брак,2,f,пенсионер,0,158616,сыграть свадьбу,свадьба


**Категоризация  по уровню дохода**

У каждого клиента уникальный доход, скорее всего в таблице даже нет одинаковых значений по этой метрике. Поэтому мы условно разделим всех клиентов на три категории: по высокому, среднему и низкому доходам. Создадим отдельный столбец income_level, в котором и будем отображать уровень заработка каждого клиента.

In [27]:
def choose_income_level(income):
    if income < 80000:
        return 'низкая'
    elif 80000 < income < 150000:
        return 'средняя'
    else:
        return 'высокая'
    
# функция принимает значение и возвращает строку(одну из категорий уровня заработка) в соответствии
# с логическим условием

data['income_level'] = data['total_income'].apply(choose_income_level)

# с помощью метода reindex() поставим столбец income_level рядом с total_income для наглядности

right_sequence = ['children', 'days_employed', 'dob_years', 'education', 'education_id', 'family_status', 'family_status_id',
                  'gender', 'income_type', 'debt', 'total_income', 'income_level', 'purpose', 'purpose_category', ]
data = data.reindex(columns=right_sequence)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,income_level,purpose,purpose_category
0,1,-8437,NaN,высшее,2,женат / замужем,1,f,сотрудник,0,253875,высокая,покупка жилья,недвижимость
1,1,-4024,NaN,среднее,1,женат / замужем,1,f,сотрудник,0,112080,средняя,приобретение автомобиля,автомобиль
2,0,-5623,NaN,среднее,1,женат / замужем,1,m,сотрудник,0,145885,средняя,покупка жилья,недвижимость
3,3,-4124,NaN,среднее,1,женат / замужем,1,m,сотрудник,0,267628,высокая,дополнительное образование,образование
4,0,340266,NaN,среднее,1,гражданский брак,2,f,пенсионер,0,158616,высокая,сыграть свадьбу,свадьба


### Вывод

Мы создали два новых столбца: income_level и purpose_category, которые улучшили читабельность и информативность таблицы с данными. Также переопределили значения столбцов education_id и family_status_id. 

Выявленные категории, для каждого из этих столбцов, помогут лучше и быстрее анализировать данные и проверять гипотезы.

<a id="step3"></a>
## Шаг 3. Ответы на вопросы

<a id="question1"></a>
### Есть ли зависимость между наличием детей и возвратом кредита в срок?

Для начала создадим новую таблицу из столбцов, подходящих для этой цели. Так же добавим новый столбец presence_child, в котором 'no' означает остутсвие детей у клиента, а 'yes' - есть хотябы один ребенок. 

In [28]:
def presence_child(child):           
    if child == 0:
        return 'no'
    else:
        return 'yes'
#методом concat объудиняем две Series в датафрейм
data_child = pd.concat([data['children'], data['debt']], axis=1)

# с помощью созданной функци presence_child создаем новый столбец, который заполняется
# в зависимости от значений столбца "children"
data_child['presence_child'] = data['children'].apply(presence_child)
    
data_child.head()

,children,debt,presence_child
0,1,0,yes
1,1,0,yes
2,0,0,no
3,3,0,yes
4,0,0,no


Теперь построим таблицу, в которой покажем отношение клиентов у которых не было просрочек по кредитам к тем, у кого были. Предварительно разделим их на две группы - те у кого есть и у кого нет детей. Так же добавим новый столбец probability_of_return, который покажет сколько процентов заемщиков возвращает кредит в срок.

In [29]:
data_child_pivot = data_child.pivot_table(index='presence_child', columns='debt', aggfunc='count')


data_child_pivot['probability_of_return'] = (data_child_pivot['children'][0] /
                         (data_child_pivot['children'][0] + data_child_pivot['children'][1]) * 100).astype('int')

data_child_pivot

children       probability_of_return
debt                  0     1                      
presence_child                                     
no                13025  1063                    92
yes                6570   669                    90

Из таблицы видно, что отсутствие детей у клиента, хоть и немного, но повышает шансы вернуть кредит в срок

Построим нову сводную таблицу, отображающую количество клиентов с разным количеством детей, сгруппированных на тех кто вовремя отдает кредит и тех, кто просрочивает.

In [30]:
data_child_pivot2 = data_child.pivot_table(index='children', columns='debt', aggfunc='count')
data_child_pivot2

presence_child        
debt                  0       1
children                       
0               13025.0  1063.0
1                4364.0   444.0
2                1857.0   194.0
3                 303.0    27.0
4                  37.0     4.0
5                   9.0     NaN

Появилось одно отсутвующее значение, значит такого наблюденя не было и мы его можем заменить на 0. Так же добавим новый столбец 'probability_of_return', который покажет сколько процентов заемщиков возвращает кредит в срок.

In [31]:
data_child_pivot2 = data_child_pivot2.fillna(0).astype('int')

data_child_pivot2['probability_of_return'] = (data_child_pivot2['presence_child'][0] /
                         (data_child_pivot2['presence_child'][0] + data_child_pivot2['presence_child'][1]) * 100).astype('int')

data_child_pivot2

presence_child       probability_of_return
debt                  0     1                      
children                                           
0                 13025  1063                    92
1                  4364   444                    90
2                  1857   194                    90
3                   303    27                    91
4                    37     4                    90
5                     9     0                   100

### Вывод

Учитывая что количество клиентов с пятью детьми минимально, их результатом мы можем принебречь. В основном количество детей никак не влияет на возвращение кредита в срок. Разве что стоит обратить внимание на результат клиентов с тремя детьми. Это как раз то количество, с которого семья в России начинает считаться многодетной и им становятся доступны определенные льготы.  

Скорее всего нагрузка в качестве каждого нового ребенка уравновешиавется разными социальными благами, такими как материнский капитал или социальная помощь многодетным семьям. Также стоит учитывать, что завести трех и более детей могут позволить состоявшиеся люди, которые уверены в своих финансах.

<a id="question2"></a>
### Есть ли зависимость между семейным положением и возвратом кредита в срок?

Для начала посморим какие варианты семейных положений существуют.

In [32]:
data['family_status'].value_counts()


женат / замужем          12259
гражданский брак          4132
не женат / не замужем     2796
в разводе                 1189
вдовец / вдова             951
Name: family_status, dtype: int64

Все значения разные и поэтому в группировке не нуждаются.

Построим таблицу из нужных нам для этого исследования столбоцов.

In [33]:
data_family = pd.concat([data['family_status'] ,data['family_status_id'], data['debt']], axis=1)
data_family.head()

,family_status,family_status_id,debt
0,женат / замужем,1,0
1,женат / замужем,1,0
2,женат / замужем,1,0
3,женат / замужем,1,0
4,гражданский брак,2,0


Теперь сгруппируем ее так, чтобы наглядно показать как соотносятя разные семейные положения с ввозвращение кредита в срок. Для большей наглядности создадим дополнительный столбец probability_of_return, который будет отображать процент вернувших в срок.

In [34]:
data_family_pivot = data_family.pivot_table(index='family_status', columns='debt', aggfunc='count')


data_family_pivot['probability_of_return'] = (data_family_pivot['family_status_id'][0] /
                         (data_family_pivot['family_status_id'][0] + data_family_pivot['family_status_id'][1]) * 100).astype('int')
data_family_pivot

family_status_id      probability_of_return
debt                                 0    1                      
family_status                                                    
в разводе                         1105   84                    92
вдовец / вдова                     888   63                    93
гражданский брак                  3747  385                    90
женат / замужем                  11332  927                    92
не женат / не замужем             2523  273                    90

### Вывод

Результат показал нам, что зависимость между семейным положением и возвратом кредита в срок существует: 

- Худшие показатели здесь у тех, кто состоит в гражданском браке или не женат / не замужем. Здесь напрашивается вывод: люди не готовые узаконить свои отношения имеют шаткое финансовое состояние. Банку следует с осторожность выдавать им кредит.

- Луший показатель у категории 'вдовец / вдова'. Клиенты, потерявшие супруга, в одночасье получили средства, принадлежащие двум дюдям. Это помогает им долгое время сохранять устойчивость в финансах. Этой категории клиентов банк может доверить свои средства.

- Средние показатели у тех кто женат / замужем или в разводе. В большинстве своем это взрослые состоявшиеся люди, который умеют планировать свой бюджет. Банку стоит обратить внимание на другие метрики при оценке возможности выдачи кредита.

<a id="question3"></a>
### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Сначала создадим новую таблицу состоящую из столбцов, подходящих для исследования.

In [35]:
data_income = pd.concat([data['total_income'], data['income_level'], data['debt']], axis=1)

data_income.head()

,total_income,income_level,debt
0,253875,высокая,0
1,112080,средняя,0
2,145885,средняя,0
3,267628,высокая,0
4,158616,высокая,0


Теперь перегруппируем данные в сводную таблицу, которая бы показывала количество людей вернувших кредит в сорк и с просрочкой по разным уровням дохода. Так же добавим новый столбец 'probability_of_return', который покажет сколько процентов заемщиков возвращает кредит в срок.

In [36]:
data_income_pivot = data_income.pivot_table(index='income_level', columns='debt', aggfunc='count')

data_income_pivot['probability_of_return'] = (data_income_pivot['total_income'][0] /
                         (data_income_pivot['total_income'][0] + data_income_pivot['total_income'][1]) * 100).astype('int')
data_income_pivot

total_income      probability_of_return
debt                    0    1                      
income_level                                        
высокая              9016  759                    92
низкая               2091  174                    92
средняя              8488  799                    91

### Вывод

Результат показал один очивидный факт - клиентам с высоким уровнем доходов можно доверять как заемщикам, у них почти всегда найдутся деньги на возврат средств вовремя.

А еще выявился один неочивидный факт - у клиентов с низким доходом показатеть на одном уровне с клиентами с высоким доходом. Это наталкивает на мысль: люди с низким уровнем дохода более рассчетливые. Возможные санкции банка из-за просрочек могут нанести им серьезный финансовый урон, который те, вероятно, не смогут сразу невелировать. Этим людям банк должен охотнее выдавать кредиты.

С клиентами со средним уровнем дохода банку стоит быть осторожнее и смотреть на другие метрики.

<a id="question4"></a>
### Как разные цели кредита влияют на его возврат в срок?

Сначала создадим новую таблицу из нужных столбцов

In [37]:
data_purpose = pd.concat([data['purpose'], data['purpose_category'], data['debt']], axis=1)
data_purpose.head()

,purpose,purpose_category,debt
0,покупка жилья,недвижимость,0
1,приобретение автомобиля,автомобиль,0
2,покупка жилья,недвижимость,0
3,дополнительное образование,образование,0
4,сыграть свадьбу,свадьба,0


Теперь сгруппируем таблицу так, чтобы она наглядно показывала количества клиентов имеющих задолжность и нет по разным целям. Также добавим новый столбец probability_of_return, которой покажет сколько процентов людей кажддой категории возвращает кредит в срок.

In [38]:
data_purpose_pivot = data_purpose.pivot_table(index='purpose_category', columns='debt', aggfunc='count')

data_purpose_pivot['probability_of_return'] = (data_purpose_pivot['purpose'][0] /
                         (data_purpose_pivot['purpose'][0] + data_purpose_pivot['purpose'][1]) * 100).astype('int')

data_purpose_pivot 

purpose      probability_of_return
debt                   0    1                      
purpose_category                                   
автомобиль          3879  400                    90
недвижимость        9399  745                    92
образование         3619  369                    90
ремонт               569   35                    94
свадьба             2129  183                    92

### Вывод

Из результата можно сделать следующие выводы:

- Категория 'ремонт' дала лучшие показатели по возврату в срок. Мало кто начинает ремонт спонтанно, люди часто готовятся и просчитывают что и в каких количествах им нужно. Кредит выдавать стоит.

- Наихудшими оказались категории 'автомобиль' и 'образование'. Машина это дорогое удовольствие, а хорошая машина - еще большее. Видимо некоторые автовладельцы не справляются с финансовым обеспечением своего авто. С образованием кажется все банальнее: преимущественно люди берут кредит на эту цель, чтобы поменять сферу деятельности. С надеждой сразу обрести новый источник дохода они встречаются с реалиями рынка. Недостаточный опыт и знания и/или безработица приводят их невозможности своевременно выплачивать кредит. Кредит стоит выдавать с особой осторожностью.

- 'Свадьба' и 'недвижимость' дали средние показатели. У кого-то хватает свадебных подарков на возврат кредита - кто-то тратит все на свадебные путешествия, у кого-то операции с жильем проходят успешно - кто-то строит дом, который рушится через неделю. В этих категориях банку следует обращать внимание на другие метрики.

<a id="end"></a>
### Шаг 4. Общий вывод

Входные данные от банка состояли из наблюдений за клиентами по десяти метрикам, характеризущих их платёжеспособность.

Во время предобработки были выявлены следующие *проблемы*: 
- отстутсвующие значения
- повторяющиеся наблюдения
- строковые значения не приведенные к одному регистру
- некоторые данные невозможно правильно прочитать без дополнительной документации
- некоторые данные принимали значения, которые невозможны в реальном мире
- некоторые числовые данные соответствовали вещественному типу данных

Каждая из проблем была обработана соответствующими методами и исправлена по принципу улучшения качества последеющего анализа.
К существующей таблице были добавлены несколько столбцов, которые как повышали читаемость и информативность таблицы, так и помогли при дальнейших исследованиях. 

Для ответов на поставленные вопросы были созданны отдельные таблицы, состоящие только из нужных для конкретного исследования данных. Ключевая метрика в них отображалась в процентом отношении.

**Исследование факторов, влияющих на несвоевременный возврат кредита в срок дало следующие результаты:**
- Количество детей практически никак не вляет на возврат долга в срок, однако клиентам у которых вообще нет детей банк может с большей уверенностью предоставить кредит.
- Клиентам побывавшим в браке или находящимся в нем более банк может доверять больше, чем тем кто ни разу не узаконил свои отношения. Отдельно хочется выделить вдов/вдовцов - они надежные заемщики
- Банк может в равной степени доверять как людям с низким, так и свысоким уровнем заработка. В отоношении клиентов со средним заработком банку следует быть осторожнее и ориентировать по другим метрикам.
- Банку следует акцентировать внимание на клиентах, которые указывают образование и приобретение автомобиля как цель заема. Исследование показало что подобные заемщики самые ненадежные.

На основе проведенного анализа возможно приблизительно описать портрет идеального заемщика: это человек, который хотя бы раз был в браке, не имеющий детей, у него либо высокий, либо низкий уровень заработка и как цель заема он указывает ремонт, покупку недвижимости или свадьбу.

Банку рекомендуется учитывать результаты исследования при построении модели кредитного скоринга.

Банку рекомендуется сопровождать данные подробной технической документацией.